In [1]:
import numpy as np
import pandas as pd
import requests
import re
import json

In [2]:
URL = "https://s3.amazonaws.com/irs-form-990/201113139349301301_public.xml"

response = requests.get(URL)
with open('data.xml', 'wb') as file:
    file.write(response.content)

In [3]:
URL = "https://s3.amazonaws.com/irs-form-990/index_2013.json"

response = requests.get(URL)
with open("index_2013.json", 'wb') as f:
    f.write(response.content)

In [4]:
with open("index_2013.json") as f:
    data = json.load(f)
    data = data[list(data.keys())[0]]

In [5]:
df = pd.DataFrame.from_dict(data)
df.head(5)

,EIN,TaxPeriod,DLN,FormType,URL,OrganizationName,SubmittedOn,ObjectId,LastUpdated
0,205585992,201306,93492296006093,990EZ,https://s3.amazonaws.com/irs-form-990/20134296...,PTOK BUSH ELEMENTARY INC,2013-12-04,201342969349200609,2016-03-21T17:23:53
1,236437027,201306,93492296007043,990EZ,https://s3.amazonaws.com/irs-form-990/20134296...,SOCIETY OF THE ARTS CO ALLENTOWN ART MUSEUM,2013-12-04,201342969349200704,2016-03-21T17:23:53
2,452096099,201212,93492296007093,990EZ,https://s3.amazonaws.com/irs-form-990/20134296...,GREAT FOUNDATIONS INC,2013-12-04,201342969349200709,2016-03-21T17:23:53
3,731664822,201212,93492310012033,990EZ,https://s3.amazonaws.com/irs-form-990/20133310...,FRIENDS OF KIDS INC,2013-12-04,201333109349201203,2016-03-21T17:23:53
4,200128337,201306,93492310003043,990EZ,https://s3.amazonaws.com/irs-form-990/20134310...,LAKESHORE YOUTH ATHLETIC ASSOC,2013-12-04,201343109349200304,2016-03-21T17:23:53


In [6]:
df.shape

(261449, 9)

In [10]:
len(data)

261449

In [34]:
df.tail(5)

,EIN,TaxPeriod,DLN,FormType,URL,OrganizationName,SubmittedOn,ObjectId,LastUpdated
261444,386624867,201306,93493270000103,990,https://s3.amazonaws.com/irs-form-990/20130270...,CONSTRUCTION EDUCATION TRUST,2013-12-03,201302709349300010,2016-03-21T17:23:53
261445,203746128,201306,93493297003363,990,https://s3.amazonaws.com/irs-form-990/20131297...,SCAGO EDUCATIONAL FACILITIES CORPORATION SPART...,2013-12-03,201312979349300336,2016-03-21T17:23:53
261446,520960095,201209,93493086004043,990,https://s3.amazonaws.com/irs-form-990/20134086...,ARC OF THE DISTRICT OF COLUMBIA,2013-04-08,201340869349300404,2016-03-21T17:23:53
261447,363330171,201209,93493086004243,990,https://s3.amazonaws.com/irs-form-990/20134086...,BANK ADMINISTRATION INSTITUTE,2013-04-08,201340869349300424,2016-03-21T17:23:53
261448,541660459,201206,93493115003213,990,https://s3.amazonaws.com/irs-form-990/20131115...,ALLIANCE DEFENSE FUND,2013-05-07,201311159349300321,2016-03-21T17:23:53


In [9]:
from multiprocessing.pool import ThreadPool

In [33]:
def download_url(url):
    file_name_start = url.rfind('/') + 1
    file_name = url[file_name_start:]
    output_dir = "/home/meso/git_repo/exploring_990/data/"
    file_name = output_dir + file_name
    
    r = requests.get(url, stream=True)
    if r.status_code == requests.codes.ok:
        with open(file_name, 'wb') as f:
            f.write(r.content)
    return url

urls = []
for i in range(df.shape[0]): # testing, should be len(data) or df.shape[0]
    urls.append(data[i]['URL'])
    
results = ThreadPool(10).imap_unordered(download_url, urls)